In [7]:
from iFinDPy import *
import pandas as pd
import numpy as np
import time,datetime
from jqdatasdk import *
# auth('18665883365','Zszq#201')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.mlab as mlab
import matplotlib.ticker as ticker

In [8]:
# # THS_iFinDLogin('yhdr079','754284')

# code='000905.SH'
# dt=THS_HF(code,'close','Fill:Original','2020-04-20 09:15:00','2023-07-05 15:15:00')
# file_path='C:/Users/changwai/Desktop/TJR/data_storage/'
# dt.data.to_csv(file_path+'000905.csv')

In [9]:
# names storageC:\Users\changwai\Desktop\TJR\data_storage
names_path='C:/Users/changwai/Desktop/TJR/'
file_path='C:/Users/changwai/Desktop/TJR/data_storage/'


# df_security_etf=get_all_securities(types=['etf'], date=None)
# df_security_etf.to_csv('all_etfs.csv')

df_security_etf=pd.read_csv(names_path+'all_etfs.csv',index_col=0) 
df_security_index=pd.read_csv(names_path+'all_indexs.csv',index_col=0) 

# get code name
def get_code(num,names_data):
    code=names_data.filter(like=str(num), axis=0).index[0]
    return code


In [10]:
# data storage
# startdate=datetime(2020,6,26)
# enddate=datetime(2023,6,26)

# def save_data(code,file_path,startdate,enddate):
#     df=get_price(code, start_date=startdate, end_date=enddate, frequency='1m', fields=['close'], skip_paused=False,fq='none', count=None, panel=False, fill_paused=True)
#     df.to_csv(file_path+code+'.csv')

# code_list=['000905']

# for code in code_list:
#     code_name=get_code(code,df_security_index)
#     save_data(code_name,file_path,startdate,enddate)

In [11]:
# extract data from storage
def get_local_data(file_path,stock,startdate,enddate):
    df=pd.read_csv(file_path+str(stock)+'.csv')
    df['time']=pd.to_datetime(df['time'])
    df=df.set_index('time')
    df=df[startdate:enddate+timedelta(days=1)]
    return df



def process_data(index,startdate,enddate):
    df=get_local_data(file_path,index,startdate,enddate)

    # data processing
    #delete first and last minute
    df=df.between_time("09:31", "14:59")

    #import last trading date of each forward
    closingdate=pd.read_excel(file_path+'closingdate.xlsx',sheet_name='lastDate') 
    closingdate['time']=pd.to_datetime(closingdate.iloc[:,1])

    closingdate_season=pd.read_excel(file_path+'closingdate.xlsx',sheet_name='lastDate_season') 
    closingdate_season['time']=pd.to_datetime(closingdate_season.iloc[:,1])

    df['ref_time']=df.index
    df['ref_time']=df['ref_time'].apply(lambda x:x.strftime('%Y-%m-%d'))
    df['ref_time']=pd.to_datetime(df['ref_time'])

    
#IC00 01
    for i in range(len(closingdate)-2):
        df.loc[(df.ref_time >= closingdate.loc[i,'time']+datetime.timedelta(1)) & (df.ref_time <= closingdate.loc[i+1,'time']+datetime.timedelta(1)),['leftdate_IC00']]=(closingdate.loc[i+1,'time']-df.ref_time).map(lambda x:x.days)
        df.loc[(df.ref_time >= closingdate.loc[i,'time']+datetime.timedelta(1)) & (df.ref_time <= closingdate.loc[i+1,'time']+datetime.timedelta(1)),['closedate_IC00']]=closingdate.loc[i+1,'time'].strftime('%m')
        df.loc[(df.ref_time >= closingdate.loc[i,'time']+datetime.timedelta(1)) & (df.ref_time <= closingdate.loc[i+1,'time']+datetime.timedelta(1)),['leftdate_IC01']]=(closingdate.loc[i+2,'time']-df.ref_time).map(lambda x:x.days)
        df.loc[(df.ref_time >= closingdate.loc[i,'time']+datetime.timedelta(1)) & (df.ref_time <= closingdate.loc[i+1,'time']+datetime.timedelta(1)),['closedate_IC01']]=closingdate.loc[i+2,'time'].strftime('%m')
#IC02 03
    for i in range(len(closingdate_season)-2):
        df.loc[(df.ref_time >= closingdate_season.loc[i,'time']+datetime.timedelta(1)) & (df.ref_time <= closingdate_season.loc[i+1,'time']+datetime.timedelta(1)),['leftdate_IC02']]=(closingdate_season.loc[i+1,'time']-df.ref_time).map(lambda x:x.days)
        df.loc[(df.ref_time >= closingdate_season.loc[i,'time']+datetime.timedelta(1)) & (df.ref_time <= closingdate_season.loc[i+1,'time']+datetime.timedelta(1)),['leftdate_IC03']]=(closingdate_season.loc[i+2,'time']-df.ref_time).map(lambda x:x.days)
        df.loc[(df.ref_time >= closingdate_season.loc[i,'time']+datetime.timedelta(1)) & (df.ref_time <= closingdate_season.loc[i+1,'time']+datetime.timedelta(1)),['closedate_IC02']]=closingdate_season.loc[i+1,'time'].strftime('%m')
        df.loc[(df.ref_time >= closingdate_season.loc[i,'time']+datetime.timedelta(1)) & (df.ref_time <= closingdate_season.loc[i+1,'time']+datetime.timedelta(1)),['closedate_IC03']]=closingdate_season.loc[i+2,'time'].strftime('%m')

        
#IC04    
    for i in range(len(closingdate_season)-3):
        df.loc[(df.ref_time >= closingdate_season.loc[i,'time']+datetime.timedelta(1)) & (df.ref_time <= closingdate_season.loc[i+1,'time']+datetime.timedelta(1)),['leftdate_IC04']]=(closingdate_season.loc[i+3,'time']-df.ref_time).map(lambda x:x.days)
        df.loc[(df.ref_time >= closingdate_season.loc[i,'time']+datetime.timedelta(1)) & (df.ref_time <= closingdate_season.loc[i+1,'time']+datetime.timedelta(1)),['closedate_IC04']]=closingdate_season.loc[i+3,'time'].strftime('%m')
        
        
    df['leftdate_IC02_copy']=df.leftdate_IC02
    df['closedate_IC02_copy']=df.closedate_IC02
    df.loc[(df.leftdate_IC02_copy==df.leftdate_IC00)|(df.leftdate_IC02_copy==df.leftdate_IC01),['leftdate_IC02']]=df['leftdate_IC03']
    df.loc[(df.leftdate_IC02_copy==df.leftdate_IC00)|(df.leftdate_IC02_copy==df.leftdate_IC01),['leftdate_IC03']]=df['leftdate_IC04']
    df.loc[(df.leftdate_IC02_copy==df.leftdate_IC00)|(df.leftdate_IC02_copy==df.leftdate_IC01),['closedate_IC02']]=df['closedate_IC03']
    df.loc[(df.leftdate_IC02_copy==df.leftdate_IC00)|(df.leftdate_IC02_copy==df.leftdate_IC01),['closedate_IC03']]=df['closedate_IC04']
    #calculate basis

    IC00=pd.read_csv(file_path+'IC00.csv')
    IC00['time']=pd.to_datetime(IC00['time'])
    IC00=IC00.set_index('time')
    IC00=IC00.drop(columns=['Unnamed: 0','thscode'])
    IC00.columns = ['IC00']

    IC01=pd.read_csv(file_path+'IC01.csv')
    IC01['time']=pd.to_datetime(IC01['time'])
    IC01=IC01.set_index('time')
    IC01=IC01.drop(columns=['Unnamed: 0','thscode'])
    IC01.columns = ['IC01']

    IC02=pd.read_csv(file_path+'IC02.csv')
    IC02['time']=pd.to_datetime(IC02['time'])
    IC02=IC02.set_index('time')
    IC02=IC02.drop(columns=['Unnamed: 0','thscode'])
    IC02.columns = ['IC02']

    IC03=pd.read_csv(file_path+'IC03.csv')
    IC03['time']=pd.to_datetime(IC03['time'])
    IC03=IC03.set_index('time')
    IC03=IC03.drop(columns=['Unnamed: 0','thscode'])
    IC03.columns = ['IC03']

    df=df.join(IC00,how='left')
    df=df.join(IC01,how='left')
    df=df.join(IC02,how='left')
    df=df.join(IC03,how='left') 

    name_list=['IC00','IC01','IC02','IC03']

    for col_name in name_list:
        df['basis'+col_name]=df[col_name]-df.close
        df['annualized_basis_'+col_name]=df['basis'+col_name]*365/(df.close*df['leftdate_'+col_name])
    df['annualized_diff_00-01']=(df['IC00']-df['IC01'])*365/(df.close*abs(df['leftdate_IC00']-df['leftdate_IC01']))
    df['annualized_diff_02-03']=(df['IC02']-df['IC03'])*365/(df.close*abs(df['leftdate_IC02']-df['leftdate_IC03']))
    return df
    

In [20]:
def plot_forth(df, y1, y1_1,y2_1,y3_1,y4_1, code_num,code_num_list,date):
  
  
  N = len(df)
  ind = np.arange(N)# the evenly spaced plot indices
  def format_date(a, pos=None):
    #保证下标不越界,很重要,越界会导致最终plot坐标轴label无显示
    thisind = np.clip(int(a+0.5), 0, N-1)
    return df.index[thisind].strftime('%Y-%m-%d')
  
  def to_percent(temp, position):
    return '%1.0f'%(10*temp) + '%'
  
  fig, ax = plt.subplots(2,2,figsize=(20,12))
  fig.suptitle(str(code_num)+'_'+date.strftime('%Y-%m'),fontsize=15)
  ax1=ax[0,0]
  ax2=ax[0,1]
  ax3=ax[1,0]
  ax4=ax[1,1]
  color=['red','blue','green','yellow']

  #ax1
  ax1.set_title(code_num_list[0])
  ax1.set_xlabel('date')
  ax1.set_ylabel('index', color=color[0])
  ax1.plot(ind, y1, color=color[0])
  ax1.tick_params(axis='y', labelcolor=color[0])
  ax1_1 = ax1.twinx()  # 创建共用x轴的第二个y轴
  ax1_1.set_ylabel('basis', color=color[1])
  ax1_1.plot(ind, y1_1, color=color[1])
  ax1_1.tick_params(axis='y', labelcolor=color[1])

  #ax2
  ax2.set_title(code_num_list[1])
  ax2.set_xlabel('date')
  ax2.set_ylabel('index', color=color[0])
  ax2.plot(ind, y1, color=color[0])
  ax2.tick_params(axis='y', labelcolor=color[0])
  ax2_1 = ax2.twinx()  # 创建共用x轴的第二个y轴
  ax2_1.set_ylabel('basis', color=color[1])
  ax2_1.plot(ind, y2_1, color=color[1])
  ax2_1.tick_params(axis='y', labelcolor=color[1])

  #ax3
  ax3.set_title(code_num_list[2])
  ax3.set_xlabel('date')
  ax3.set_ylabel('index', color=color[0])
  ax3.plot(ind, y1, color=color[0])
  ax3.tick_params(axis='y', labelcolor=color[0])
  ax3_1 = ax3.twinx()  # 创建共用x轴的第二个y轴
  ax3_1.set_ylabel('basis', color=color[1])
  ax3_1.plot(ind, y3_1, color=color[1])
  ax3_1.tick_params(axis='y', labelcolor=color[1])

  #ax4
  ax4.set_title(code_num_list[3])
  ax4.set_xlabel('date')
  ax4.set_ylabel('index', color=color[0])
  ax4.plot(ind, y1, color=color[0])
  ax4.tick_params(axis='y', labelcolor=color[0])
  ax4_1 = ax4.twinx()  # 创建共用x轴的第二个y轴
  ax4_1.set_ylabel('basis', color=color[1])
  ax4_1.plot(ind, y4_1, color=color[1])
  ax4_1.tick_params(axis='y', labelcolor=color[1])


  ax1.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
  ax2.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
  ax3.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
  ax4.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
  ax2_1.yaxis.set_major_formatter(ticker.PercentFormatter(1))
  ax3_1.yaxis.set_major_formatter(ticker.PercentFormatter(1))
  ax4_1.yaxis.set_major_formatter(ticker.PercentFormatter(1))
  
  fig.autofmt_xdate()
  
  fig.tight_layout()
  return fig

def plot_second(df, y1, y1_1,y2_1, code_num,code_num_list,date):
  N = len(df)
  ind = np.arange(N)# the evenly spaced plot indices
  
  
  def format_date(a, pos=None):
    #保证下标不越界,很重要,越界会导致最终plot坐标轴label无显示
    thisind = np.clip(int(a+0.5), 0, N-1)
    return df.index[thisind].strftime('%Y-%m-%d')
  
  def to_percent(temp, position):
    return '%1.0f'%(10*temp) + '%'
  
  fig, ax = plt.subplots(1,2,figsize=(20,6))
  fig.suptitle(str(code_num)+'_'+date.strftime('%Y-%m'),fontsize=15)
  ax1=ax[0]
  ax2=ax[1]
  color=['red','blue','green','yellow']

  #ax1
  ax1.set_title(code_num_list[0])
  ax1.set_xlabel('date')
  ax1.set_ylabel('index', color=color[0])
  ax1.plot(ind, y1, color=color[0])
  ax1.tick_params(axis='y', labelcolor=color[0])
  ax1_1 = ax1.twinx()  # 创建共用x轴的第二个y轴
  ax1_1.set_ylabel('basis', color=color[1])
  ax1_1.plot(ind, y1_1, color=color[1])
  ax1_1.tick_params(axis='y', labelcolor=color[1])

  #ax2
  ax2.set_title(code_num_list[1])
  ax2.set_xlabel('date')
  ax2.set_ylabel('index', color=color[0])
  ax2.plot(ind, y1, color=color[0])
  ax2.tick_params(axis='y', labelcolor=color[0])
  ax2_1 = ax2.twinx()  # 创建共用x轴的第二个y轴
  ax2_1.set_ylabel('basis', color=color[1])
  ax2_1.plot(ind, y2_1, color=color[1])
  ax2_1.tick_params(axis='y', labelcolor=color[1])


  ax1.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
  ax2.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
  ax2.yaxis.set_major_formatter(ticker.FuncFormatter(to_percent))
  fig.autofmt_xdate()
  
  fig.tight_layout()
  return fig

def plot_all(df):

    fig1=plot_forth(df,df.close,df.basisIC00,df.basisIC01,df.basisIC02,df.basisIC03,code_num='basis',code_num_list=['IC00','IC01','IC02','IC03'])
    fig2=plot_forth(df,df.close,df.annualized_basis_IC00,df.annualized_basis_IC01,df.annualized_basis_IC02,df.annualized_basis_IC03,code_num='annualized_basis',code_num_list=['IC00','IC01','IC02','IC03'])
    fig3=plot_forth(df,df.close,df.basisIC00-df.basisIC02,df['annualized_diff_00-01'],df.close,df.close,code_num='diff01',code_num_list=['IC00-IC01','IC00-IC01(%)','IC02','IC03'])
    fig4=plot_forth(df,df.close,df.basisIC02-df.basisIC03,df['annualized_diff_02-03'],df.close,df.close,code_num='diff03',code_num_list=['IC02-IC03','IC02-IC03(%)','IC02','IC03'])
    return[fig1,fig2,fig3,fig4]

def time_points(df,num):
    df_points=df.groupby(df.index.to_period('D') ).apply(lambda x: x.iloc[::num,:])
    df_points.index=df_points.index.droplevel(0)
    df_points=df_points.groupby(df_points.index.to_period('D') ).apply(lambda x: x.iloc[1: ,:])
    df_points.index=df_points.index.droplevel(0)
    return df_points

In [21]:
#最基础的4个图
# startdate=datetime.datetime(2020,1,1)
# enddate=datetime.datetime(2023,7,1)

# date_bins=pd.date_range(startdate,enddate,freq='M',normalize=True)

# for st in date_bins:
#   index='000905'
#   df=process_data(index,st,st)
#   df_points=time_points(df,30)
#   if len(df)==0:
#     continue
#   else:
#     sum = PdfPages(st.strftime('%Y-%m-%d')+'.pdf')
#     figures=plot_all(df)
#     sum.savefig(figures[0])
#     sum.savefig(figures[1])
#     sum.savefig(figures[2])
#     sum.savefig(figures[3])
#     sum.close()
    
#   plt.cla()
#   plt.close("all")


In [22]:
#仅IC02-03的图，基差+年化
# startdate=datetime.datetime(2020,1,1)
# enddate=datetime.datetime(2023,7,1)
# date_bins=pd.date_range(startdate,enddate,freq='M',normalize=True)

# interval=30
# backforce=30

# sum = PdfPages('IC02-03.pdf')

# for st in date_bins:
#   index='000905'
#   df=process_data(index,st-datetime.timedelta(days=backforce),st)
#   df['annualized_diff_00-01']=(df['IC00']-df['IC01'])*365/(df.close*abs(df['leftdate_IC00']-df['leftdate_IC01']))
#   df['annualized_diff_02-03']=(df['IC02']-df['IC03'])*365/(df.close*abs(df['leftdate_IC02']-df['leftdate_IC03']))

#   if len(df)==0:
#     continue
#   else:
#     df_points=time_points(df,interval)
#     # sum = PdfPages(st.strftime('%Y-%m')+'.pdf')
#     figures=plot_second(df_points,df_points.close,df_points.basisIC02-df_points.basisIC03,
#                         df_points['annualized_diff_02-03'],code_num='basis',code_num_list=['IC02-03','IC02-03(annualized)'],date=st)
#     sum.savefig(figures)
#   plt.cla()
#   plt.close("all")
# sum.close()




In [23]:
#仅IC02 - 03的图，4个图分别 basis anul IC02anual IC03annual
startdate=datetime.datetime(2020,4,20)
enddate=datetime.datetime(2023,7,1)
date_bins=pd.date_range(startdate,enddate,freq='3M',normalize=True)-datetime.timedelta(days=12)

interval=30
forward=90

closedate_list=['03','06','09','12']

time0=closedate_list[1]
time1=closedate_list[2]

sum = PdfPages(time0+'-'+time1+'.pdf')

for st in date_bins:
  index='000905'
  df=process_data(index,st,st+datetime.timedelta(days=forward))
  df=df.loc[(df.closedate_IC02==time0)&(df.closedate_IC03==time1),:]
  df['annualized_diff_02-03']=(df['IC02']-df['IC03'])*365/(df.close*abs(df['leftdate_IC02']-df['leftdate_IC03']))

  if len(df)==0:
    continue
  else:
    df_points=time_points(df,interval)
    # sum = PdfPages(st.strftime('%Y-%m')+'.pdf')
    figures=plot_forth(df_points,df_points.close,df_points.basisIC02-df_points.basisIC03,
                        df_points['annualized_diff_02-03'],df_points.annualized_basis_IC02,df_points.annualized_basis_IC03,code_num='basis',code_num_list=['IC02-03','IC02-03(annualized)','IC02(annualized)','IC03(annualized)'],date=st)
    sum.savefig(figures)
  plt.cla()
  plt.close("all")
sum.close()


In [ ]:
#仅IC00 - 03 06 09 12 的图，4个图分别 basis anul IC00anual IC03annual
startdate=datetime.datetime(2020,4,20)
enddate=datetime.datetime(2023,7,1)
date_bins=pd.date_range(startdate,enddate,freq='3M',normalize=True)-datetime.timedelta(days=12)

interval=30
forward=90

closedate_list=['03','06','09','12']

time0=closedate_list[1]
time1=closedate_list[2]

sum = PdfPages('IC00'+'-'+time1+'.pdf')

for st in date_bins:
  index='000905'
  df=process_data(index,st,st+datetime.timedelta(days=forward))
  df=df.loc[(df.closedate_IC02==time0)&(df.closedate_IC03==time1),:]

  df['annualized_diff_00-03']=(df['IC00']-df['IC03'])*365/(df.close*abs(df['leftdate_IC00']-df['leftdate_IC03']))
  if len(df)==0:
    continue
  else:
    df_points=time_points(df,interval)
    # sum = PdfPages(st.strftime('%Y-%m')+'.pdf')
    figures=plot_forth(df_points,df_points.close,df_points.basisIC00-df_points.basisIC03,
                        df_points['annualized_diff_00-03'],df_points.annualized_basis_IC00,df_points.annualized_basis_IC03,code_num='basis',code_num_list=['IC02-03','IC02-03(annualized)','IC02(annualized)','IC03(annualized)'],date=st)
    sum.savefig(figures)
  plt.cla()
  plt.close("all")
sum.close()


In [ ]:
startdate=datetime.datetime(2022,10,1)
enddate=datetime.datetime(2022,11,1)
df=process_data('000905',startdate,enddate)

df['annualized_diff_02-03']=(df['IC02']-df['IC03'])*365/(df.close*abs(df['leftdate_IC02']-df['leftdate_IC03']))
# plot_all(df)

df_points=time_points(df,30)
plot_forth(df_points,df_points.close,df_points.basisIC02-df_points.basisIC03,df_points['annualized_diff_02-03'],
           df_points.basisIC02,df_points.basisIC03,code_num='basis',code_num_list=['IC02-IC03','IC','IC02','IC03'])


In [ ]:
df_points